In [16]:
# Pull rep from zipcode

import mechanize

# Insantiate browser
br = mechanize.Browser()
br.set_handle_robots(False)

# Open lookup link
br.open("https://ziplook.house.gov/")

# Find form and input zipcode
br.select_form(nr=1)
br.form["ZIP"] = "22182"
br.submit()


print(br.geturl())

https://ziplook.house.gov/htbin/findrep_house?ZIP=22182


In [ ]:
# Scrape for rep name

import requests